# Unit 2: Adapt URDF for Gazebo Simulator

URDF files are very useful, as you have seen, for having a virtual representation of the different links and joints.<br>
But you are not simulating its weight, its inertia, what sensors it has, how it collides with other objects, the friction with the floor, or how the server position control will react to the robot. These are just some of the topics that you are going to learn about in this unit.

## 1. Add Collisions:

One of the most crucial elements in being able to simulate a robot is how it interacts with the world around it.<br>
At the moment, your URDF Mira Robot would be a Ghost in a simulation. There is a visual representation, but it can't interact with anything. It will go through the floor and the objects.<br>
So, the first thing you have to do is add collisions to your URDF model. Here you have an example for the base_link:

As you can see, the only difference is that there is a new tag called <b>collision</b> that specifies the collision geometry. This is the shape used for calculating the physical contacts.<br>
As you might have guessed, you can also use 3D meshes here, like this:

This is not advised, as the physics calculations are more intensive as the mesh gets more complex.<br>
That's why the collisions are normally basic geometric shapes, while the visuals are meshes.<br>
Another alternative if the geometry of the contact is crucial, is to use a lower poly version of the virtual mesh. That way, the shape is maintained, but less calculation power is needed.

## 2. Spawn a robot in Gazebo through URDF Files

Now, how do you spawn a URDF file-defined robot into the simulated world?<br>
Through these two launch files:

This first one's name is <b>spawn_urdf.launch</b>:

This first launch spawns the given URDF file into the given point in space, if a gazebo simulation is running.<br>
You can call this first launch through a second launch, passing the necessary arguments to it:

This second one's name is <b>spawn_mira.launch</b>:

<p style="background:#EE9023;color:white;">Exercise U2-1</p>

Create the two previously mentioned launches to spawn your <b>mira.urdf</b> URDF file.

Then, add all of the collisions to the links that need to have them.<br>
This means that the roll_M1_link, pitch_M2_link, yaw_M3_link, camera_link, and right_eye_link/left_eye_link <b>don't</b> need them.<br> Why?<br>
The eyes and camera don't because they are too small or don't protrude to have any significant effect on collisions. As for the roll, pitch, and yaw, they don't have to have collisions because they are virtual links; they don't exist in reality and it would just cause problems that the real robot doesn't have.

Finally, spawn your URDF and see what happens.

<p style="background:#EE9023;color:white;">END Exercise U2-1</p>

<p style="background:#E80000;color:white;">Warning</p>

You might have seen nothing appear, or so it seems.<br>
Execute the following command to have a list of all the models inside the simulation:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosservice call /gazebo/get_world_properties "{}"<br>
</th>
</tr>
</table>

So it <b>is</b> in the simulation, but it's not represented.<br>
That's because Gazebo needs a minimum of the <b>INERTIAS</b> of the object to have it represented, and a <b>VISUAL</b> to know how to represent it. The collisions only define how it will collide, but doesn't give any mass data, vital for a physical simulation in Gazebo.<br>
This <b>doesn't happen</b> in <b>SDF</b> files. The inertias are automatically introduced and you can have a robot model with only the collisions and visuals.

<p style="background:#E80000;color:white;">Warning</p>

Don't forget to remove the model from the Gazebo simulation before trying to spawn a new version again. Otherwise, Gazebo won't allow you to spawn a new one.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosservice call /gazebo/delete_model "model_name: 'mira'"<br>
</th>
</tr>
</table>

Or you can make your life easier by creating a client that removes the model:

In [ ]:
#! /usr/bin/env python

import rospy
from gazebo_msgs.srv import DeleteModel, DeleteModelRequest # Import the service message used by the service /gazebo/delete_model
import sys 

rospy.init_node('remove_model_service_client') # Initialise a ROS node with the name service_client
print "Waiting for Service /gazebo/delete_model"
rospy.wait_for_service('/gazebo/delete_model') # Wait for the service client /gazebo/delete_model to be running
delete_model_service = rospy.ServiceProxy('/gazebo/delete_model', DeleteModel) # Create the connection to the service
kk = DeleteModelRequest() # Create an object of type DeleteModelRequest
kk.model_name = "mira" # Fill the variable model_name of this object with the desired value
print "Deleting model ="+str(kk)
status_message = delete_model_service(kk) # Send through the connection the name of the object to be deleted by the service
print status_message

You can execute it like so:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun my_mira_description remove_mira.py<br>
</th>
</tr>
</table>

It's possible that <b>none of these systems will work</b> because there was an error in the spawn or something similar. If this happens, you can remove the model just by <b>right-clicking on it and selecting Delete</b>.

But if you can't <i>/gazebo/delete_model</i>, the gazebo has probably died. In that case, you should reboot your gazebo, or in RobotIgniteAcademy, just return to Unit 0 and come back to this unit again. That will restart everything.

## 3. Add Inertias:

Here is an example of how the inertias would be added to the base_link:

Here are some elements to comment on:

The Inertial tag has:<br>
<ul>
<li><b>origin</b>: As you already know, this will move the inertia element based on the axis frame of the base_link. For example, it's used for lowering the center of mass of the link element, if needed.</li>
<li><b>mass</b>: It's very important that the weight of this link element in Kg is coherent.<br>
The reason is that this will condition the dynamics and the power of the actuators used afterwards to move the joints. But because it is not always possible to get the weight of each part of a robot, it at least has to be proportional to the total weight, which is much easier to get.</li>
<li><b>inertia</b>: As it states, this is the inertia matrix of the link. It will condition how the link behaves when spinning and moving around.<br>
There could be a whole course on inertia calculations. But for you, as the user, you will always assume that the material the link is made of is homogeneous, and the matrix is diagonal. This means that you will only have values in the ixx, iyy, and izz. They will always be calculated based on primary geometric shapes.<br>
This link has the basic inertia moments : https://en.wikipedia.org/wiki/List_of_moments_of_inertia</li>
</ul>

Although this way of calculating the Inertial tag data might seem poor, bear in mind that by approximating inertias to basic shapes, the results are quite close to reality. And if you really need a perfect simulation, you will have to dedicate time to obtaining the exact data from the real model.

Here you have a simple python script created to calculate the three basic inertia moments based on the mass and different variables.

In [ ]:
#!/usr/bin/env python
import math


class InertialCalculator(object):

    def __init__(self):
        print "InertialCalculator Initialised..."

    def start_ask_loop(self):

        selection = "START"

        while selection != "Q":
            print "#############################"
            print "Select Geometry to Calculate:"
            print "[1]Box width(w)*depth(d)*height(h)"
            print "[2]Sphere radius(r)"
            print "[3]Cylinder radius(r)*height(h)"
            print "[Q]END program"
            selection = raw_input(">>")
            self.select_action(selection)

        print "InertialCaluclator Quit...Thank you"

    def select_action(self, selection):
        if selection == "1":
            mass = float(raw_input("mass>>"))
            width = float(raw_input("width>>"))
            depth = float(raw_input("depth>>"))
            height = float(raw_input("height>>"))
            self.calculate_box_inertia(m=mass, w=width, d=depth, h=height)
        elif selection == "2":
            mass = float(raw_input("mass>>"))
            radius = float(raw_input("radius>>"))
            self.calculate_sphere_inertia(m=mass, r=radius)
        elif selection == "3":
            mass = float(raw_input("mass>>"))
            radius = float(raw_input("radius>>"))
            height = float(raw_input("height>>"))
            self.calculate_cylinder_inertia(m=mass, r=radius, h=height)
        elif selection == "Q":
            print "Selected Quit"
        else:
            print "Usage: Select one of the give options"


    def calculate_box_inertia(self, m, w, d, h):
        Iw = (m/12.0)*(pow(d,2)+pow(h,2))
        Id = (m / 12.0) * (pow(w, 2) + pow(h, 2))
        Ih = (m / 12.0) * (pow(w, 2) + pow(d, 2))
        print "BOX w*d*h, Iw = "+str(Iw)+",Id = "+str(Id)+",Ih = "+str(Ih)

    def calculate_sphere_inertia(self, m, r):
        I = (2*m*pow(r,2))/5.0
        print "SPHERE Ix,y,z = "+str(I)

    def calculate_cylinder_inertia(self, m, r, h):
        Ix = (m/12.0)*(3*pow(r,2)+pow(h,2))
        Iy = Ix
        Iz = (m*pow(r,2))/2.0
        print "Cylinder Ix,y = "+str(Ix)+",Iz = "+str(Iz)

if __name__ == "__main__":
    inertial_object = InertialCalculator()
    inertial_object.start_ask_loop()

This will speed up the process of obtaining the inertias of all the links you work on in this course.

And there is also a "gazebo" tag that refers to the link that is setting the physical properties:

Here the main physical properties of the link are defined:<br>
<ul>
<li>mu1: The static friction coefficient. In simple terms, it would be how much friction there is until the object starts moving.</li>
<li>mu2: The dynamic friction coefficient. In simple terms, it would be how much friction there is when the object is moving.</li>

</ul>

Here we have the same issue. These values should be calculated through friction tests with elements with the same mass as the links you are setting these values to. You should also bear in mind the materials they are made of, and so on. But in reality, it's setting them with the values that makes the robot behave correctly, not necessarily the real ones.

<ul>
<li>kp: This coefficient sets the static contact stiffness. This determines if the link material is closer to marble (very rigid, bigger values) or more like rubber (soft material, lower values).</li>
<li>kp: This coefficient sets the dynamic contact stiffness. This determines if the link material is closer to marble (very rigid, bigger values) or more like rubber (soft material, lower values). It's essentially how much it deforms over a long period of time, exerting its pressure.</li>
</ul>

In Mira Robot's case, this is not as relevant as with Gurdy Robot, where you will work a bit longer on where locomotion depends on friction with the floor.

For further details, this Gazebo page gives you all the options for the Gazebo tag. Go to the section on <b>"Elements For Links"</b>: http://gazebosim.org/tutorials/?tut=ros_urdf

<p style="background:#EE9023;color:white;">Exercise U2-2</p>

Add all of the inertial tags for all the links. Use the python programs to calculate the inertias of each link.<br>
All of the inertia calculations will be based on the geometry of the collision geometries.<br>
Here you have the weight of all the links:<br>
<ul>
<li>base_link: mass value="0.18"</li>
<li>roll_M1_link: value="0.001"</li>
<li>pitch_M2_link: value="0.001"</li>
<li>yaw_M3_link: value="0.001"</li>
<li>head_link: mass value="0.02"</li>
<li>left_eye_link: mass value="0.001"</li>
<li>right_eye_link: mass value="0.001"</li>
<li>camera_link: mass value="0.001"</li>
</ul>

The links with mass set to <b>0.001</b> are just symbolic mass that has no real effect on the dynamics of the robot, but allow it to be represented in Gazebo.

<p style="background:#EE9023;color:white;">END Exercise U2-2</p>

<p style="background:green;color:white;">Solution Exercise U2.2</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

When spawning, you should get a perfectly rendered Mira Robot, though it will be limp. Because we haven't set up any friction in the joints, it will continue to move with the residual spawn movement forever.

<img src="img/mira_nocontrollers.gif"/>

Your URDF should be something similar to this:

<p style="background:#E80000;color:white;">Warning</p>

Don't forget to remove the model from the Gazebo simulation before trying to spawn a new version again. Otherwise, Gazebo won't allow you to spawn a new one.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosservice call /gazebo/delete_model "model_name: 'mira'"<br>
</th>
</tr>
</table>

If this doesn't work, which sometimes happens in Gazebo, just right-click and hit DELETE.

## 4. Add controllers:

As you have experienced in the previous exercise, Mira's joints don't have any effort applied and, therefore, they fall due to their weight. And we can't move them through the joints publisher gui, as you should know, because that's only a topic that indicates the sensor readings of the encoders inside the actuators (virtual or physical).<br>
So, you have to add real actuators to move Mira's joints. These are the steps for a single joint, in this case the "roll_joint":

### Step 1: Add a transmission tag linked to the joint you want to move:

Tags explained:

First, let's have a look at the joint limits:

As you can see, there is a maximum effort of 0.1, and a maximum velocity of 0.005. This is how fast the joint will be able to go and how much effort they will allow the actuator to apply. This is like selecting the kind of server that you want. Select a server that can move the weight of that link and the ones connected to it down the tree, but don't select a huge server because it will just break your robot, or simply make it fly due to the inertia forces. Bear this in mind because it's very easy to get carried away in virual design, but then the robot behaves in the most bizarre way.

This activates the gazebo control plugin for the Mira namespace. This namespace will be set in the main spawn launch file.

Here you define the name of the transmission, which will have to be unique through the URDF file.

The type of transmission that, for the moment, is the only one implemented: "transmission_interface/SimpleTransmission"

Here you link the joint to the transmission and select the hardware interface, which also only has an effort interface working.

Here you set a unique name for the actuator and again select the hardware interface. As for the reduction, it's self-explanatory.

All these values are almost always the same, so don't worry too much because, in most cases, putting these values is more than enough.

### Step 2: Create a Control config yaml to set the PID for each transmission/motor

Create a file called <b>mira_control.yaml</b> inside the <b>config</b> folder of your package, <b>my_mira_description</b>.

Here you will set the publish rate of the joints position, that is now controlled by the joint_state_controller.<br>
And you set the joint controllers:<br>
<ul>
<li>The name of the controller is recommended to have the exact same joint name, proceeded by "_position_controller"</li>
<li>Then, you select the type, which we recomment as the JointPositionController because we want to control it through the position of the joints.</li>
<li>The JointName: This is linking the joint to the controller.</li>
<li>pid: Sets the PID values. This is absolutely vital to making the actuators reach the desired position without oscillations or going too slow.</li>
</ul>

If you are uncertain of how PID work, although it's a deep subject, this tutorial is magnificent for giving a practical idea of how to diagnose PID settings and how to use them: https://www.flitetest.com/articles/p-i-and-sometimes-d-gains-in-a-nutshell

Also, you will set a sensible PID at the start, so you can then calibrate it during the execution of movements through the DynamicReconfigure http://wiki.ros.org/dynamic_reconfigure and its GUI, rqt_reconfigure http://wiki.ros.org/rqt_reconfigure.<br>
Through rqt_reconfigure, you can change any parameter that is made accesible at runtime. This way, you just have to move the robot and change the values until you see that it behaves in the correct way.

To use it, you will just have to execute the following command when you spawn Mira with its controllers:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun rqt_reconfigure rqt_reconfigure<br>
</th>
</tr>
</table>

### Step 3: Create the control launch file that gets everything running

Now, it's time to turn on the switch for the controllers and the actuators. For this, you have to create the following launch file, called <b>mira_control.launch</b>:

<b>Be careful with name spaces; they're a source of errors.</b>

If you don't understand what's going on in this launch, please refer to our course on <b>TF_ROS</b> for a more detailed explanation.<br>
Esentially here we are loading the <b>yaml file</b> and launching the controllers with the robot state publisher for the TF publication.

This part is the only one that is really relevant. Here you state that all the controllers that you defined in the yaml are active. In this case, you are only turning on the "roll_joint_position_controller" and the "joint_state_controller." When adding more controllers, just add them here.<br>

### Step 4: Spawn and launch the controllers in their own name space:

The last step is creating a launch that spawns the Mira Robot in its own name space and starts the controllers.<br>
Spawning in its own name space allows it to be spawned alongside more robots using the urdf_spawner and other systems.

You have to define the <b>"spawn_with_controllers_mira.launch"</b> that will do the real work.

These two files <b>(spawn_mira.launch and mira_control.launch)</b> you already have, so now you just have to launch your <b>"spawn_with_controllers_mira.launch."</b> It should spawn your Mira URDF model, with the roll joint as fully controllable.

To test that the controllers are working, you can:
<ul>
<li>Publish directly in the controllers topics:</li>
<li>Publish through the rqt_gui Topic Publishing.</li>
</ul>

To publish directly, pick a controller command topic; in this case, there must be only one; and publish a position inside the limits of that joint (in this case lower="-0.2" upper="0.2"): 

Mira Robot should move its head in the roll direction.

To publish through the gui, just execute:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun rqt_gui rqt_gui<br>
</th>
</tr>
</table>

<img src="img/rqtgui.png"/>

<img src="img/rqtgui_2.png"/>

Introducing a sinus based on the time (i) and divided by the frequency, Mira should move its head side to side. So, because the yaw is not yet controlled, it continues to move freely.

<img src="img/mira_oneconstroller.gif"/>

<p style="background:#EE9023;color:white;">Exercise U2-3</p>

Add two more transmissions; one for the pitch_joint and the other for the yaw_joint.<br>
When you have it working by commands, create a python script that moves Mira.<br>
Try to make a simple set of movements and try to mimic how it moves in reality.<br>
Remember that if you find that the movements are too fast or oscillate too much, you can always use the rqt_reconfigure to tune the PID values.<br>

<p style="background:#EE9023;color:white;">END Exercise U2-3</p>

<p style="background:#E80000;color:white;">Warning</p>

If you have to remove the robot to restart again, due to a gazebo issue, you have to delete the model by RIGHT-clicking and hitting Delete. Don't forget to also close the control launch.

<p style="background:green;color:white;">Solution Exercise U2.3</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

This is how your URDF should look:

And the launch control file should be similar to this:

And the yaml file should be:

And here is an example of how a moving python script could be done and what movement it should generate:

<img src="img/mira_solo.gif"/>

In [1]:
#!/usr/bin/env python
import time
import rospy
from math import pi, sin, cos, acos
import random
from std_msgs.msg import Float64
from sensor_msgs.msg import JointState
"""
Topics To Write on:
type: std_msgs/Float64
/mira/pitch_joint_position_controller/command
/mira/roll_joint_position_controller/command
/mira/yaw_joint_position_controller/command
"""

class MiraJointMover(object):

    def __init__(self):
        rospy.init_node('jointmover_demo', anonymous=True)
        rospy.loginfo("Mira JointMover Initialising...")
        self.pub_mira_roll_joint_position = rospy.Publisher('/mira/roll_joint_position_controller/command',
                                                            Float64,
                                                            queue_size=1)
        self.pub_mira_pitch_joint_position = rospy.Publisher('/mira/pitch_joint_position_controller/command',
                                                             Float64,
                                                             queue_size=1)
        self.pub_mira_yaw_joint_position = rospy.Publisher('/mira/yaw_joint_position_controller/command',
                                                           Float64,
                                                           queue_size=1)
        joint_states_topic_name = "/mira/joint_states"
        rospy.Subscriber(joint_states_topic_name, JointState, self.mira_joints_callback)
        mira_joints_data = None
        while mira_joints_data is None:
            try:
                mira_joints_data = rospy.wait_for_message(joint_states_topic_name, JointState, timeout=5)
            except:
                rospy.logwarn("Time out " + str(joint_states_topic_name))
                pass

        self.mira_joint_dictionary = dict(zip(mira_joints_data.name, mira_joints_data.position))

    def move_mira_all_joints(self, roll, pitch, yaw):
        angle_roll = Float64()
        angle_roll.data = roll
        angle_pitch = Float64()
        angle_pitch.data = pitch
        angle_yaw = Float64()
        angle_yaw.data = yaw
        self.pub_mira_roll_joint_position.publish(angle_roll)
        self.pub_mira_pitch_joint_position.publish(angle_pitch)
        self.pub_mira_yaw_joint_position.publish(angle_yaw)

    def move_mira_roll_joint(self, position):
        """
        limits radians : lower="-0.2" upper="0.2"
        :param position:
        :return:
        """
        angle = Float64()
        angle.data = position
        self.pub_mira_roll_joint_position.publish(angle)

    def move_mira_pitch_joint(self, position):
        """
        limits radians : lower="0" upper="0.44"
        :param position:
        :return:
        """
        angle = Float64()
        angle.data = position
        self.pub_mira_pitch_joint_position.publish(angle)

    def move_mira_yaw_joint(self, position):
        """
        Limits : continuous, no limits
        :param position:
        :return:
        """
        angle = Float64()
        angle.data = position
        self.pub_mira_yaw_joint_position.publish(angle)

    def mira_joints_callback(self, msg):
        """
        sensor_msgs/JointState
        std_msgs/Header header
        uint32 seq
        time stamp
        string frame_id
        string[] name
        float64[] position
        float64[] velocity
        float64[] effort

        :param msg:
        :return:
        """
        self.mira_joint_dictionary = dict(zip(msg.name, msg.position))

    def mira_check_joint_value(self, joint_name, value, error=0.1):
        """
        Check the joint by name 'pitch_joint', 'roll_joint', 'yaw_joint' is near the value given
        :param value:
        :return:
        """
        similar = self.mira_joint_dictionary.get(joint_name) >= (value - error ) and self.mira_joint_dictionary.get(joint_name) <= (value + error )

        return similar

    def convert_angle_to_unitary(self, angle):
        """
        Removes complete revolutions from angle and converts to positive equivalent
        if the angle is negative
        :param angle: Has to be in radians
        :return:
        """
        # Convert to angle between [0,360)
        complete_rev = 2 * pi
        mod_angle = int(angle / complete_rev)
        clean_angle = angle - mod_angle * complete_rev
        # Convert Negative angles to their corresponding positive values
        if clean_angle < 0:
            clean_angle += 2 * pi

        return clean_angle

    def assertAlmostEqualAngles(self, x, y,):
        c2 = (sin(x) - sin(y)) ** 2 + (cos(x) - cos(y)) ** 2
        angle_diff = acos((2.0 - c2) / 2.0)
        return angle_diff

    def mira_check_continuous_joint_value(self, joint_name, value, error=0.1):
        """
        Check the joint by name 'pitch_joint', 'roll_joint', 'yaw_joint' is near the value given
        We have to convert the joint values removing whole revolutions and converting negative versions
        of the same angle
        :param value:
        :return:
        """
        joint_reading = self.mira_joint_dictionary.get(joint_name)
        clean_joint_reading = self.convert_angle_to_unitary(angle=joint_reading)
        clean_value = self.convert_angle_to_unitary(angle=value)

        dif_angles = self.assertAlmostEqualAngles(clean_joint_reading, clean_value)
        similar = dif_angles <= error

        return similar

    def mira_movement_sayno(self):
        """
        Make Mira say no with the head
        :return:
        """
        check_rate = 5.0
        position = 0.7


        rate = rospy.Rate(check_rate)
        for repetition in range(2):
            similar = False
            while not similar:
                self.move_mira_yaw_joint(position=position)
                # WARNING: THE COMMAND HAS TO BE PUBLISHED UNTIL THE GOAL IS REACHED
                # This is because, when publishing a topic, the first time doesn't always work.
                similar = self.mira_check_continuous_joint_value(joint_name="yaw_joint", value=position)

                rate.sleep()
            position *= -1



    def mira_movement_look(self, roll, pitch, yaw):
        """
        Make Mira look down
        :return:
        """
        check_rate = 5.0
        position_roll = roll
        position_pitch = pitch
        position_yaw = yaw

        similar_roll = False
        similar_pitch = False
        similar_yaw = False
        rate = rospy.Rate(check_rate)
        while not (similar_roll and similar_pitch and similar_yaw):
            self.move_mira_all_joints(position_roll, position_pitch, position_yaw)
            similar_roll = self.mira_check_continuous_joint_value(joint_name="roll_joint", value=position_roll)
            similar_pitch = self.mira_check_continuous_joint_value(joint_name="pitch_joint", value=position_pitch)
            similar_yaw = self.mira_check_continuous_joint_value(joint_name="yaw_joint", value=position_yaw)
            rate.sleep()


    def mira_lookup(self):

        self.mira_movement_look(roll=0.0,pitch=0.3,yaw=1.57)

    def mira_lookdown(self):

        self.mira_movement_look(roll=0.0,pitch=0.0,yaw=1.57)

    def mira_movement_laugh(self, set_rpy=False, roll=0.05, pitch=0.0, yaw=1.57, n_giggle=15):
        """
        Giggle in a given pitch yaw configuration
        :return:
        """
        position_roll = roll
        position_pitch = pitch
        position_yaw = yaw
        for repetitions in range(n_giggle):
            if set_rpy:
                self.move_mira_all_joints(position_roll, position_pitch, position_yaw)
            else:
                self.move_mira_roll_joint(position_roll)
            time.sleep(0.1)
            position_roll *= -1

    def mira_moverandomly(self):
        roll = random.uniform(-0.15, 0.15)
        pitch = random.uniform(0.0, 0.3)
        yaw = random.uniform(0.0, 2*pi)
        self.mira_movement_look(roll, pitch, yaw)

    def movement_random_loop(self):
        """
        Executed movements in a random way
        :return:
        """
        rospy.loginfo("Start Moving Mira...")
        while not rospy.is_shutdown():
            self.mira_moverandomly()
            self.mira_movement_laugh()


if __name__ == "__main__":
    mira_jointmover_object = MiraJointMover()
    mira_jointmover_object.movement_random_loop()



ModuleNotFoundError: No module named 'rospy'

Basically, this class publishes in the controller topics <b>( /mira/roll_joint_position_controller/command, /mira/pitch_joint_position_controller/command /mira/yaw_joint_position_controller/command )</b> and checks the <b>/mira/joint_states</b> topic to make sure that the robot has arrived at the given position.<br>
It also deals with the fact that joint angles sometimes have negative or complete turn versions of the same angle, and you have to make a system similar to the one in the function "assertAlmostEqualAngles" to be sure that the angles are the same, even though they might be different versions of the same angle.

For more details:<br>
http://gazebosim.org/tutorials/?tut=ros_control<br>
http://wiki.ros.org/urdf/XML/Transmission

## 5. Adding Sensors

The last step is to add sensors to the robot. In this case, you have to add a camera where the "camera_link" is.<br>
But thanks to the gazebo plugin system, this is a plug and play operation. You just have to add the following tag to the end of the URDF file:

As you can see, there are many variables that can be changed to adapt the camera to the real specifications. But the most important variable to set is the <b>reference="camera_link,"</b> which will link the camera to the camera_link position. The <b>pose</b> tag is also important. This allows it to point the camera where it should go and not based on the camera_link axis. In this case, you can see that it's turned 90º to point in the correct direction.

To view what the camera sees, just execute rqt_image_view and select the image topic:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun rqt_image_view rqt_image_view 
</th>
</tr>
</table>

Then, open GraphicalTools:

<img src="img/font-awesome_desktop.png">

<img src="img/mira_cam.png"/>

<p style="background:#E80000;color:white;">Warning</p>

In this case, removing the robot <b>with sensors</b> will crash Gazebo in any system. This is a known issue and they are working on solving it. For you as user, when you delete a robot model with sensors, just restart your Gazebo or, in RobotIgniteCase, just go to Unit 0 and come back again, restarting all of the systems.

Another solution is executing the command "rosservice call /gazebo/reset_simulation TAB-TAB." This will kill the Gazebo process that might be a zombie. Once Gazebo is killed, RobotIgniteSystem will detect it and relaunch a fresh, new Gazebo for you.

## Congratulations! You can now create any robot from scratch and simulate it in Gazebo

In the next unit, you will learn how to create the Gurdy Robot and how to convert URDF files into XACRO format files for cleaner robot model definitions.